In [246]:
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import numpy as np
import os 
from random import shuffle
import cv2

In [247]:
class NeuralNetwork(object):
   
    # Replace 0 with your sigmoid calculation here
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate,hidden_layers,total_nodes):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.hidden_layers=hidden_layers
        self.tn=total_nodes
        self.lr = learning_rate
        self.weights=[0]*(hidden_layers+1)
        
        
        #self.computeActivation = lambda x : 1/(1 + np.exp(-x))
        self.initialize()
    def computeActivation(self,x):
        return 1/(1 + np.exp(-x))
    
    def initialize(self):
        #self.hidden_output=[0]*(hidden_layers+1)
        if self.hidden_layers==0:
            self.weights[0]=np.random.uniform(-1, 1,(self.input_nodes, self.output_nodes))
        else:
            for i in range(self.hidden_layers+1):
                if i==0:
                    self.weights[i]=np.random.uniform(-1, 1,(self.input_nodes, self.hidden_nodes))
                elif i==hidden_layers:
                    self.weights[i]=np.random.uniform(-1,1, (self.hidden_nodes, self.output_nodes))
                else:
                    self.weights[i]=np.random.uniform(-1, 1,(self.hidden_nodes, self.hidden_nodes))
       
    #def initialize(self):
       
          
    
    
    
    def compute_error(self,final_op,y):
        return y-final_op
    
    def feedForward(self,X,hidden_output):
        #print("X is ",X)
        prev_input=X
        hidden_output[0]=X
        if self.hidden_layers==0:
            next_input=np.dot(prev_input,self.weights[0])
            prev_input=next_input
            final_output=self.computeActivation(prev_input)
        else:
            for i in range(self.hidden_layers+1):
                if i!=0:
                    prev_input=self.computeActivation(prev_input)
                    hidden_output[i]=prev_input
               #print(prev_input.shape)
                #print(self.weights[i].shape)
                #print(i)
                next_input=np.dot(prev_input,self.weights[i])
                prev_input=next_input
            #print(prev_input.shape)
            
            final_output=self.computeActivation(prev_input)
        #final_output=self.computeActivation(final_output)
        #print("final_output",final_output)
        return final_output,hidden_output
    
    
    
    def computeWeightChange(self,final_output,y,X,delta_weights,hidden_output):
        kk=self.hidden_layers+1
        prev_error=0
        error=0
        prev_error=0
        for j in range(kk):
                i=kk-1-j
                if i==(kk-1):
                    error=self.compute_error(final_output,y)
                    output_error_term=error*final_output*(1-final_output)
                    prev_error=output_error_term
                    delta_weights[i]+=output_error_term*hidden_output[i][:,None]
                else:
                    hidden_error=np.dot(self.weights[i+1],prev_error)
                    prev_error=hidden_error
                    output_error_term=hidden_error*hidden_output[i+1]*(1-hidden_output[i+1])
                    delta_weights[i]+=output_error_term*hidden_output[i][:,None]
        return delta_weights
    
    def backpropogate(self,final_output,y,X,delta_weights,hidden_output):
        kk=self.hidden_layers+1
        prev_error=0
        error=0
        prev_error=0
        if self.hidden_layers==0:
            error=self.compute_error(final_output,y)
            output_error_term=error*final_output*(1-final_output)
            #prev_error=output_error_term
            delta_weights[0]+=output_error_term*hidden_output[0][:,None]
            
        else:
            delta_weights=self.computeWeightChange(final_output,y,X,delta_weights,hidden_output)
                
        return delta_weights
            
            
            
                
    def updateWeights(self,delta_weights):
        for i in range(len(self.weights)):
            self.weights[i]+=delta_weights[i]*self.lr/self.tn
    
    def train(self,X,y):
        delta_weights=[0]*(self.hidden_layers+1)
        hidden_output=[0]*(self.hidden_layers+1)
        #print("y ",y)
        final_output,hidden_output=self.feedForward(X,hidden_output)
        #print("train",final_output)
        #print("fo :" ,final_output )
        delta_weights=self.backpropogate(final_output,y,X,delta_weights,hidden_output)
        self.updateWeights(delta_weights)
        
    def train1(self,X):
        delta_weights=[0]*(self.hidden_layers+1)
        hidden_output=[0]*(self.hidden_layers+1)
        #print("y ",y)
        final_output,hidden_output=self.feedForward(X,hidden_output)
        
        #print("ff          ",final_output)
        return final_output
    

In [248]:
def label_image(img):
    f1= img.split(".")
    categ=f1[0]
    if categ=="cat":
        return 0
    elif categ=="dog":
        return 1
    else:
        return -1

In [249]:

def prepare_data(path1):
    dataW=[]
    for img in tqdm(os.listdir(path=path1)):
        #print(img)
        img_category = label_image(img)
        path_2_img = os.path.join(path1,img)
        img1 = cv2.resize(cv2.imread(path_2_img,cv2.IMREAD_GRAYSCALE),(50,50))
        img2=img1.flatten()
        dataW.append([np.array(img2),img_category])
        #print("shape",dataW[0][0].shape)
    return dataW

In [239]:
path='/home/user/Downloads/ml/train/'
training_data=prepare_data(path)
print("ooo")
t_path='/home/user/Downloads/ml/test/'
test_data=prepare_data(t_path)
print("o1")

  0%|          | 15/12500 [00:00<01:28, 141.85it/s]

ooo


100%|██████████| 12500/12500 [01:38<00:00, 127.35it/s]

o1


In [258]:
iterations = 50
learning_rate = 0.5
hidden_nodes = 128
output_nodes = 1
hidden_layers = 0
input_nodes=2500
total_nodes=len(training_data)
print(total_nodes)
k1=int((total_nodes*70)/100)
k2=total_nodes-k1
train_data=training_data[:k1]
test1_data=training_data[:-k2]

nn1=NeuralNetwork( input_nodes, hidden_nodes, output_nodes, learning_rate,hidden_layers,k1)
#nn1.initialize()
for i in range(iterations):
    print(i)
    for l in train_data:
        nn1.train(l[0],l[1])
for i in range(len(nn1.weights)):
    print(nn1.weights[i])
print("done")

25000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
[[-0.13931901]
 [ 0.85001591]
 [-0.90080765]
 ...
 [-0.36686133]
 [-0.70082295]
 [ 0.49821597]]
done


In [252]:
tp=0
tn=0
fp=0
fn=0
for l in test1_data:
    y=nn1.train1(l[0])
    #print(y)
    if y>=0.5:
        flag=1
    else:
        flag=0
   
    if l[1]==1:
        if flag==1:
                tp+=1
        else:
            fn+=1
    elif l[1]==0:
        if flag==0:
            tn+=1
        else:
            fp+=1
print("tp ",tp)
print("fp ",fp)
print("tn ",tn)
print("fn ",fn)
            

tp  6246
fp  5424
tn  3306
fn  2524


In [255]:
for l in test_data:
    y1=nn1.train1(l[0])
    if y1>=0.5:
        flag=1
    else:
        flag=0
    if flag==1:
        cate='dog'
    else:
        cate='cat'
    file1 = open("/home/user/Downloads/ml/0.txt","a")
    #append mode 
    file1.write(cate)
    file1.write("\n") 
file1.close() 
        
    

In [259]:
iterations = 50
learning_rate = 0.5
hidden_nodes = 128
output_nodes = 1
hidden_layers = 1
input_nodes=2500
total_nodes=len(training_data)
print(total_nodes)
k1=int((total_nodes*70)/100)
k2=total_nodes-k1
train_data=training_data[:k1]
test1_data=training_data[:-k2]

nn1=NeuralNetwork( input_nodes, hidden_nodes, output_nodes, learning_rate,hidden_layers,k1)
#nn1.initialize()
for i in range(iterations):
    print(i)
    for l in train_data:
        nn1.train(l[0],l[1])
for i in range(len(nn1.weights)):
    print(nn1.weights[i])
print("done")

25000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
[[-0.42074434  0.69064516 -0.311845   ... -0.23033369  0.10056219
   0.71185564]
 [ 0.03744497  0.14278161  0.91288035 ...  0.2819939   0.69946076
  -0.53036472]
 [-0.81264691 -0.02292814 -0.18994962 ...  0.35498665 -0.34743454
   0.96023442]
 ...
 [ 0.79625905  0.75019663  0.39209858 ...  0.45960431  0.17466188
   0.03728607]
 [-0.85327282  0.87555382  0.72409896 ...  0.98038115  0.46223218
   0.16591915]
 [-0.23899069  0.51189572  0.17583217 ...  0.97970316  0.66852561
  -0.01228735]]
[[ 0.56467677]
 [-0.10002402]
 [-0.73169108]
 [-0.7763421 ]
 [-0.42213941]
 [ 0.86010192]
 [ 0.26085454]
 [ 0.90689593]
 [-0.78721464]
 [-0.92561715]
 [ 0.66857999]
 [-0.34840306]
 [-0.27426765]
 [-0.44181742]
 [-0.49905795]
 [-0.40921284]
 [-0.56728477]
 [ 0.12811031]
 [-0.50926892]
 [ 0.10254264]
 [-0.00916146]
 [-0.4044865 ]
 [ 0.48294469]
 [ 1.03576039]
 [

In [260]:
iterations = 50
learning_rate = 0.5
hidden_nodes = 128
output_nodes = 1
hidden_layers = 2
input_nodes=2500
total_nodes=len(training_data)
print(total_nodes)
k1=int((total_nodes*70)/100)
k2=total_nodes-k1
train_data=training_data[:k1]
test1_data=training_data[:-k2]

nn1=NeuralNetwork( input_nodes, hidden_nodes, output_nodes, learning_rate,hidden_layers,k1)
#nn1.initialize()
for i in range(iterations):
    print(i)
    for l in train_data:
        nn1.train(l[0],l[1])
for i in range(len(nn1.weights)):
    print(nn1.weights[i])
print("done")

25000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
[[-0.35793594  0.61544153 -0.91186059 ... -0.14983859  0.79035428
  -0.37510298]
 [-0.94619744  0.493044    0.28532594 ... -0.01172524  0.85483109
   0.64886742]
 [-0.59309416 -0.91634027 -0.46353422 ... -0.25949898  0.16854035
   0.83737321]
 ...
 [ 0.6072613   0.62094096 -0.52126195 ... -0.54522339 -0.94355719
   0.9733509 ]
 [ 0.06713038  0.15433065 -0.0044503  ... -0.55249588  0.84547702
   0.77422884]
 [-0.24328482 -0.73301809 -0.16595661 ... -0.71361396 -0.43590716
   0.309346  ]]
[[ 0.47539852  0.66280736 -0.41441959 ... -0.37808984 -0.67768546
  -0.7505445 ]
 [-0.40211464  0.34471089 -0.02864825 ...  0.77357973 -0.51076452
  -0.30033551]
 [ 0.46597784  0.55800514  0.38443739 ...  0.6669818  -0.58851213
  -0.00871961]
 ...
 [ 0.40422869 -0.37564464 -0.6573755  ... -0.96351772  0.76539508
  -0.23845724]
 [-0.07568321 -0.8855659   0.5147

In [261]:
iterations = 50
learning_rate = 0.5
hidden_nodes = 128
output_nodes = 1
hidden_layers = 3
input_nodes=2500
total_nodes=len(training_data)
print(total_nodes)
k1=int((total_nodes*70)/100)
k2=total_nodes-k1
train_data=training_data[:k1]
test1_data=training_data[:-k2]

nn1=NeuralNetwork( input_nodes, hidden_nodes, output_nodes, learning_rate,hidden_layers,k1)
#nn1.initialize()
for i in range(iterations):
    print(i)
    for l in train_data:
        nn1.train(l[0],l[1])
for i in range(len(nn1.weights)):
    print(nn1.weights[i])
print("done")

25000
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
[[-0.1499955   0.37462068 -0.40328617 ... -0.63486595 -0.91096207
  -0.40051065]
 [-0.12476344 -0.31427864 -0.22500871 ...  0.26087524  0.41912412
  -0.07979161]
 [-0.86881627  0.21685074  0.45630786 ...  0.4711046  -0.61670473
  -0.87633597]
 ...
 [ 0.59870718 -0.29587011  0.07554245 ... -0.96533775  0.6068689
  -0.51402687]
 [-0.64111273  0.56478102  0.79600479 ... -0.65471453  0.93293783
  -0.33964692]
 [ 0.1043332   0.72599354 -0.67551309 ... -0.90503944 -0.73192214
  -0.86689851]]
[[-0.18057852  0.0322619   0.86994572 ... -0.11643923 -0.32918004
  -0.52291878]
 [-0.67681784  0.75665017  0.77550175 ...  0.6656242   0.32652585
   0.08372582]
 [-0.31565547  0.6382064   0.33450915 ... -0.74414458 -0.18657093
  -0.76477174]
 ...
 [-0.71390814 -0.67286997  0.64963362 ... -0.21289668  0.74042373
   0.19264613]
 [-0.93582982  0.89872829 -0.46623